<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c9fe24f9a5ca0cb6fdc200cd1c975605260c214864843eb45ffea122aa58850b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 09:22:46
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -25.42 K (-0.18%)
Current PnL: -29.56 L (-19.1%)
CY Booked + Current PnL: -15.27 L (-9.87%)
-------------------
Total profit:  1.23 L
Total loss:  -30.79 L
-------------------
Total Booked + Current PnL: 11.71 L (9.16%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.3%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 94.19 L (67.9%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.67,-17.95,21.99,0.09,18183.0,-18090.0,82687.0,91.89,35.0,M-SC,2.34,253.0,-0.99,0.62,8.63,OX40N,NTT,DURABLES
43,ITC,452.00,-0.11,-1.61,12.24,10.44,24103.0,-3218.0,196920.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.47,3.87,X40,NTT,FMCG
50,MASFIN,398.61,-0.72,-2.92,25.72,22.05,24464.0,-2865.0,95115.0,-16.44,54.0,H-SC,7.05,164.0,-0.12,0.71,38.26,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.35,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.51,OX40N,NTT,PAINTS
82,VOLTAS,1530.00,-0.40,4.48,14.56,19.69,29170.0,8598.0,200340.0,-5.78,44.0,X-MC,2.54,78.0,0.29,1.50,11.59,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.98,-21.77,92.75,50.79,102055.0,-30621.0,110032.0,0.50,68.0,H-SC,5.88,151.0,-0.30,0.82,28.11,XR,ATH,IT
41,INFY,2275.00,1.13,9.10,41.95,54.87,143207.0,28487.0,341375.0,-14.93,63.0,X-LC,3.24,2.0,0.20,2.55,18.14,X40,BTT,IT
1,ABB,7934.00,0.96,-4.57,52.53,45.57,131153.0,-11947.0,249672.0,-38.66,55.0,H-MC,5.10,120.0,-0.09,1.87,6.24,AR,NTT,ELECTRICAL
67,SONACOMS,806.63,0.78,-17.78,69.73,39.54,57994.0,-17990.0,83169.0,-35.70,41.0,M-SC,4.47,220.0,-0.31,0.62,17.28,AR,ATH,AUTO
72,TATAELXSI,9161.00,0.65,-29.46,87.17,32.03,81065.0,-38839.0,92996.0,-23.59,30.0,H-SC,8.67,148.0,-0.48,0.70,4.16,OX40N,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,HATHWAY,31.4,-1.71,-30.35,148.81,73.29,112679.0,-33000.0,75720.0,780.43,38.0,H-SC,7.51,172.0,-0.29,0.57,2.10,XR,NTT,ENTERTAINMENT
12,BERGEPAINT,680.0,-1.62,-3.76,25.88,21.14,56621.0,-8558.0,218781.0,-10.57,40.0,X-MC,4.19,74.0,-0.15,1.64,23.32,XY24,NTT,PAINTS
44,JCHAC,2282.0,-1.60,-36.78,58.16,-0.01,46992.0,-47008.0,80797.0,16443.48,23.0,M-SC,4.99,233.0,-1.00,0.60,3.41,OX40N,NTT,AC
47,KANSAINER,340.0,-1.38,-26.07,53.49,13.47,106637.0,-70308.0,199359.0,-69.67,29.0,H-SC,4.22,158.0,-0.66,1.49,1.37,XY24,NTT,PAINTS
70,SYMPHONY,1306.0,-1.36,-33.47,50.25,-0.03,57217.0,-57276.0,113865.0,-46.65,44.0,M-SC,16.81,206.0,-1.00,0.85,1.38,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.69,-6.84,123.19,107.91,169433.0,-10106.0,137538.0,-24.20,39.0,M-SC,12.36,216.0,-0.06,1.03,2.55,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.35,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.51,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.67,-17.95,21.99,0.09,18183.0,-18090.0,82687.0,91.89,35.0,M-SC,2.34,253.0,-0.99,0.62,8.63,OX40N,NTT,DURABLES
66,SIS,528.00,-0.20,-25.03,61.74,21.26,51395.0,-27787.0,83245.0,1962.69,45.0,H-SC,4.59,166.0,-0.54,0.62,12.47,OX40N,NTT,MISC
47,KANSAINER,340.00,-1.38,-26.07,53.49,13.47,106637.0,-70308.0,199359.0,-69.67,29.0,H-SC,4.22,158.0,-0.66,1.49,1.37,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.62,6.47,61.8,72.27,111492.0,10960.0,180408.0,-8.8,67.0,M-LC,1.93,99.0,0.1,1.35,13.04,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.62,6.47,61.80,72.27,111492.0,10960.0,180408.0,-8.80,67.0,M-LC,1.93,99.0,0.10,1.35,13.04,XR,NTT,IT
38,INDIAMART,4810.62,-0.26,-4.22,115.83,106.72,136832.0,-5204.0,118132.0,-52.54,30.0,H-SC,2.00,139.0,-0.04,0.88,18.17,AR,ATH,MISC
25,FINCABLES,1641.55,-0.69,-6.84,123.19,107.91,169433.0,-10106.0,137538.0,-24.20,39.0,M-SC,12.36,216.0,-0.06,1.03,2.55,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.06,-2.54,39.17,35.63,80004.0,-5326.0,204248.0,-16.30,38.0,H-MC,3.15,119.0,-0.07,1.53,13.71,AR,ATH,PHARMA
1,ABB,7934.00,0.96,-4.57,52.53,45.57,131153.0,-11947.0,249672.0,-38.66,55.0,H-MC,5.10,120.0,-0.09,1.87,6.24,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,0.45,-20.32,52.59,21.58,58724.0,-28476.0,111663.0,-59.80,18.0,X-MC,7.24,56.0,-0.48,0.83,0.45,X40N,ATH,IT
9,BAJAJHFL,181.50,-0.19,-21.44,88.96,48.45,140132.0,-42984.0,157522.0,-27.18,25.0,X-MC,12.18,64.0,-0.31,1.18,2.56,X40N,ATH,FINANCE
75,TMPV,600.00,0.06,-18.58,74.62,42.18,167938.0,-51352.0,225058.0,-26.74,25.0,X-LC,6.08,3.0,-0.31,1.68,0.06,XY24,NTT,AUTO
23,DMART,5391.45,-0.61,-9.87,42.14,28.12,79919.0,-20757.0,189650.0,-22.27,28.0,X-LC,4.98,19.0,-0.26,1.42,11.43,X40N,ATH,FMCG
53,PAGEIND,51605.08,0.26,-10.13,40.27,26.07,59261.0,-16580.0,147160.0,-33.76,28.0,X-MC,12.16,55.0,-0.28,1.10,0.26,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,0.35,-80.61,543.69,24.79,265468.0,-203029.0,48827.0,-81.54,38.0,X-SC,1.16,86.0,-0.76,0.37,19.36,X40N,NTT,MISC
8,AWL,485.0,-0.49,-22.16,97.19,53.50,228282.0,-66869.0,234882.0,-62.35,32.0,X-MC,1.73,58.0,-0.29,1.76,4.33,XY24,NTT,FMCG
43,ITC,452.0,-0.11,-1.61,12.24,10.44,24103.0,-3218.0,196920.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.47,3.87,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.09,6.47,23.01,30.97,43300.0,11441.0,188179.0,-18.62,64.0,X-MC,2.07,75.0,0.26,1.41,19.85,X40,ATH,INSURANCE
3,ACC,3906.0,-0.07,-25.15,119.31,64.15,212491.0,-59851.0,178100.0,-56.31,35.0,X-SC,2.10,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.07,-25.15,119.31,64.15,212491.0,-59851.0,178100.0,-56.31,35.0,X-SC,2.10,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,-0.16,-6.04,42.35,33.76,79914.0,-12120.0,188700.0,-33.72,33.0,X-MC,5.23,57.0,-0.15,1.41,0.00,X40,ATH,FMCG
75,TMPV,600.00,0.06,-18.58,74.62,42.18,167938.0,-51352.0,225058.0,-26.74,25.0,X-LC,6.08,3.0,-0.31,1.68,0.06,XY24,NTT,AUTO
31,HAVELLS,2069.16,0.24,-10.80,48.52,32.48,141956.0,-35419.0,292572.0,-15.33,36.0,X-MC,5.52,67.0,-0.25,2.19,0.24,X40,ATH,ELECTRICAL
53,PAGEIND,51605.08,0.26,-10.13,40.27,26.07,59261.0,-16580.0,147160.0,-33.76,28.0,X-MC,12.16,55.0,-0.28,1.10,0.26,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.33,-30.75,106.27,42.84,47841.0,-19988.0,45018.0,-54.82,39.0,X-SC,36.57,83.0,-0.42,0.34,1.85,XY24,NTT,MISC
15,CAMS,4762.00,0.35,-80.61,543.69,24.79,265468.0,-203029.0,48827.0,-81.54,38.0,X-SC,1.16,86.0,-0.76,0.37,19.36,X40N,NTT,MISC
59,RELAXO,1176.00,-0.96,-47.19,188.41,52.31,144429.0,-68503.0,76657.0,-43.76,44.0,X-SC,6.41,91.0,-0.47,0.57,2.61,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.80,-37.77,117.27,35.20,93898.0,-48600.0,80070.0,4.98,35.0,X-SC,15.66,92.0,-0.52,0.60,1.71,X40,NTT,FOOTWEAR
22,DIXON,18931.72,0.45,-20.32,52.59,21.58,58724.0,-28476.0,111663.0,-59.80,18.0,X-MC,7.24,56.0,-0.48,0.83,0.45,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.49,-12.92,36.98,19.28,111406.0,-44695.0,301261.0,-23.95,61.0,X-LC,5.20,1.0,-0.40,2.25,11.37,X40,ATH,IT
41,INFY,2275.00,1.13,9.10,41.95,54.87,143207.0,28487.0,341375.0,-14.93,63.0,X-LC,3.24,2.0,0.20,2.55,18.14,X40,BTT,IT
75,TMPV,600.00,0.06,-18.58,74.62,42.18,167938.0,-51352.0,225058.0,-26.74,25.0,X-LC,6.08,3.0,-0.31,1.68,0.06,XY24,NTT,AUTO
81,VBL,671.64,-0.57,-5.01,42.83,35.67,128499.0,-15822.0,300020.0,-16.24,52.0,X-LC,4.61,4.0,-0.12,2.24,8.15,X40N,ATH,FMCG
43,ITC,452.00,-0.11,-1.61,12.24,10.44,24103.0,-3218.0,196920.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.47,3.87,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.23,-17.60,120.47,81.67,93126.0,-16508.0,77302.0,6944.44,41.0,L-SC,13.86,271.0,-0.18,0.58,49.70,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.72,-2.92,25.72,22.05,24464.0,-2865.0,95115.0,-16.44,54.0,H-SC,7.05,164.0,-0.12,0.71,38.26,XR,ATH,FINANCE
13,BSOFT,831.70,1.98,-21.77,92.75,50.79,102055.0,-30621.0,110032.0,0.50,68.0,H-SC,5.88,151.0,-0.30,0.82,28.11,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.35,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.51,OX40N,NTT,PAINTS
36,ICICIPRULI,790.00,-0.09,6.47,23.01,30.97,43300.0,11441.0,188179.0,-18.62,64.0,X-MC,2.07,75.0,0.26,1.41,19.85,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.98,-21.77,92.75,50.79,102055.0,-30621.0,110032.0,0.50,68.0,H-SC,5.88,151.0,-0.30,0.82,28.11,XR,ATH,IT
84,WIPRO,420.00,0.62,6.47,61.80,72.27,111492.0,10960.0,180408.0,-8.80,67.0,M-LC,1.93,99.0,0.10,1.35,13.04,XR,NTT,IT
32,HCLTECH,1922.01,0.53,8.69,14.76,24.73,38811.0,21020.0,262944.0,13.61,65.0,X-LC,6.22,7.0,0.54,1.97,25.25,X40,ATH,IT
56,RAJESHEXPO,518.00,-0.35,-63.38,173.25,0.07,87034.0,-86941.0,50236.0,1740.91,54.0,L-SC,2.34,268.0,-1.00,0.38,24.94,OX40N,NTT,JEWELLERY
5,ASIANPAINT,3460.25,-0.25,-2.22,23.69,20.95,58322.0,-5579.0,246189.0,-3.16,49.0,X-LC,18.69,31.0,-0.10,1.84,32.63,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.80
1,25,45.83
2,50,76.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.95
MC    30.34
LC    24.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.25
X40      22.56
X40N     10.81
XR       10.10
XY25      9.41
AR        9.35
OX40N     7.77
SR        1.02
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.45
X-MC    23.54
X-LC    21.13
M-SC    12.09
X-SC     6.98
H-MC     5.03
L-SC     1.43
M-MC     1.41
M-LC     1.35
H-LC     1.18
L-LC     1.06
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.39,-8.24,43.24
IT,12.80,-17.62,81.22
FINANCE,9.56,-21.18,72.39
ELECTRICAL,6.26,-12.15,53.23
PAINTS,6.09,-13.38,30.22
MISC,6.06,-60.11,125.71
INSURANCE,4.65,-1.69,37.53
PHARMA,4.12,-1.53,34.03
AUTO,3.39,-26.44,80.65


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3257667.0,21
AR,1320643.0,10
XR,1311455.0,13
X40,1021855.0,14
X40N,1002080.0,9
OX40N,752505.0,10
XY25,383841.0,6
SR,283895.0,2
MH,85431.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3706178.0,25
M-SC,1440654.0,15
X-MC,1334725.0,16
X-SC,987460.0,8
X-LC,935561.0,11
H-MC,407191.0,3
L-SC,271031.0,3
M-LC,111492.0,1
H-LC,75121.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1296030.0      6
           AR         905649.0      5
M-SC       XY24       820776.0      6
H-SC       XR         786074.0      7
X-SC       X40N       507130.0      3
X-MC       X40        461399.0      7
           XY24       399040.0      3
X-LC       X40        375849.0      5
H-SC       OX40N      349099.0      4
M-SC       OX40N      316372.0      5
X-SC       XY24       295723.0      3
X-MC       X40N       286532.0      4
H-SC       SR         283895.0      2
X-LC       XY24       250064.0      2
H-MC       AR         211157.0      2
X-LC       X40N       208418.0      2
H-MC       XY24       196034.0      1
X-MC       XY25       187754.0      2
X-SC       X40        184607.0      2
L-SC       XR         183997.0      2
M-SC       XR         174790.0      2
           AR         128716.0      2
M-LC       XR         111492.0      1
X-LC       XY25       101230.0      2
L-SC       OX40N       87034.0      1
H-SC       MH          85431.0      1
H-LC       AR          75121.0      1
M-MC       XY25        56897.0      1
L-MC       XR          55102.0      1
L-LC       XY25        37960.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
